In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from model import generate_synthetic_trajectories, pack_model_params
from em_algorithm import em_viterbi_optimization

### Generate a dataframe with the trajectory data of 10 particles

#### For experimental data, please load a df which includes columns particle,t,x,y and declare dt, N_steps and N_particles, as well as an initial guess for the four model parameters, then define model_params = pack_model_params(T_stick,T_unstick,D,A,dt)

In [ ]:
T_stick, T_unstick, D, A = 100, 100, 1, 1
dt = 10
T = 1e3
N_steps = int(T/dt)
N_particles = 10
df, model_params = generate_synthetic_trajectories(N_steps,N_particles,dt,T_stick,T_unstick,D,A,
                                                   random_seed=1)

### Run the algorithm over all particles and apply bootstrap procedure

In [ ]:
N_bootstrap = 10
alg_output_list = []
bootstrap_estimates_list = []
for n in range(N_particles):
    X_arr = df.loc[df.particle==n,["x","y"]].values
    alg_output_list.append(em_viterbi_optimization(X_arr,model_params,dt,verbose=False))        
    
    #Apply bootstrap correction: generate synthetic trajectories based on the algorithm's output    
    output_params = alg_output_list[n][0][-1]
    if np.isnan(output_params).any():
        bootstrap_estimates_list.append(np.array([np.nan]*4))
    else:
        bootstrap_output = []
        for m in range(N_bootstrap):
            model_params_for_bootstrap = pack_model_params(*output_params,dt)
            df_bootstrap,model_params_bootstrap = generate_synthetic_trajectories(N_steps,N_particles,dt,*output_params,random_seed=None)
            bootstrap_output.append(em_viterbi_optimization(df_bootstrap[["x","y"]].values,model_params_bootstrap,dt,verbose=False)[0][-1])
        bootstrap_output = np.array(bootstrap_output)
        bootstrap_estimates_list.append(2*output_params-np.median(bootstrap_output,axis=0)) # consider mean instead of median

### Visualize the trajectories (darker shade is tethered)

In [ ]:
fig,ax=plt.subplots(figsize=(7,7))
colors = plt.cm.get_cmap("tab10").colors
for particle,particle_df in df.groupby("particle"):
    bright_color = np.array(colors[particle % len(colors)])
    dark_color = bright_color/2
    cmap = LinearSegmentedColormap.from_list("my_cmap", (bright_color,dark_color), N=2)
    xy = particle_df[["x","y"]].values.reshape(-1,1,2)
    segments = np.hstack([xy[:-1], xy[1:]])
    coll = LineCollection(segments, cmap=cmap)
    state_est = alg_output_list[particle][4].flatten() #true_state = df.state.values
    coll.set_array(state_est)
    ax.add_collection(coll)

    
ax.autoscale_view()
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_aspect("equal")

### Compare the true hidden states versus the algorithm's prediction

In [ ]:
fig,ax=plt.subplots(N_particles//2,2,sharex=True,sharey=True,figsize=(14,10))
for n in range(N_particles):
    cur_ax = ax[n//2,n%2]
    cur_ax.plot(df.loc[df.particle==n,"state"].values,label="True S")
    cur_ax.plot(alg_output_list[n][4].flatten(),"--",label="Est S")
    cur_ax.set_title(n)
    if n%2 == 0:
        cur_ax.set_ylabel("S")
    if n//2 == N_particles//2:
        cur_ax.set_xlabel("t")
leg=ax[0,0].legend()

### Compare the estimated model parameters with the true model parameters

In [ ]:
fig,ax=plt.subplots(2,2)
params_str = ["T0","T1","D","A"]
true_params = [T_stick, T_unstick, D, A]
for n_param in range(4):
    cur_ax = ax[n_param//2,n_param%2]
    sns.histplot([alg_output_list[n][0][-1][n_param] for n in range(N_particles)], ax=cur_ax)
    cur_ax.set_ylabel("")
    cur_ax.set_xlabel(params_str[n_param])
    cur_ax.plot(true_params[n_param]*np.ones(2),cur_ax.get_ylim(),"k--",linewidth=2)
fig.tight_layout()